# Analysis of the Combined data for 7pad Scanning

In [2]:
from single_pad_analysis.analysis_functions import *
from single_pad_analysis.HexDetector import HexDetector

In [3]:
run_number = 325
basedir_path = '/home/akallits/Documents/PicoAnalysis/Saclay_Analysis/data/2023_April_h4/processedTrees/ParameterTrees/'
get_run_file_path = lambda pool_num: f'{basedir_path}Run{run_number}-Pool{pool_num}_treeParam.root'
pool_number_dict = { #key:value
    2: 0,
    3: 0,
    4: 0,
    5: 0 }

In [ ]:
for pool_number in pool_number_dict.keys():
    file_path = get_run_file_path(pool_number)
    tree = get_tree(file_path, 'ParameterTree')
    mcp_channel = 'C1'
    mm_channel = 'C2'
    channels = [mcp_channel, mm_channel]
    global_vars = ['eventNo', 'chi2track']
    channel_vars = ['hitX', 'hitY']
    peak_param_channel_vars = ['ampl', 'dampl', 'charge', 'echarge', 'echargefit', 'echargefixed', 'totcharge', 'totchargefixed', 'risetime', 'tfit20','tfit20_nb', 'tnaive20', 'sigmoidR[4]']

    branches = []
    for channel in channels:
        for channel_var in channel_vars:
            branches.append(f'{channel_var}_{channel}')
        for var in peak_param_channel_vars:
            branches.append(f'peakparam_{channel}/peakparam_{channel}.{var}')
    for var in global_vars:
        branches.append(var)
    df = get_df_branches(tree, branches)

    print(f'Start processing: {df["eventNo"].shape[0]} events')
    get_best_tracks(df, channels, chi2_quality, plot=True)

    for channel in channels:
        get_single_peak(df, channel)

    event_start_number = get_run_event_start(df)
    print(event_start_number)
    # event_start, event_end = 4416, None
    event_start, event_end = event_start_number, None

    if event_start is not None:
        df = df[df['eventNo'] >= event_start]
    if event_end is not None:
        df = df[df['eventNo'] <= event_end]

    time_col = 'tfit20_nb'
    df[f'{mm_channel}_time_diff'] = df[f'peakparam_{mm_channel}/peakparam_{mm_channel}.{time_col}'] - df[f'peakparam_{mcp_channel}/peakparam_{mcp_channel}.{time_col}']

    filters = {channel: {} for channel in channels}
    for channel in channels:

        tot_charge_data = df[f'peakparam_{channel}/peakparam_{channel}.totcharge']
        filters[channel].update({'totcharge_cut': (tot_charge_data > min_charge_tot) & (tot_charge_data < max_charge_tot)})

        echarge_data = df[f'peakparam_{channel}/peakparam_{channel}.echarge']
        filters[channel].update({'echarge_cut': (echarge_data > min_echarge) & (echarge_data < max_echarge)})

        echarge_fit_data = df[f'peakparam_{channel}/peakparam_{channel}.echargefit']
        filters[channel].update({'echargefit_cut': (echarge_fit_data > min_echarge) & (echarge_fit_data < max_echarge)})

        charge_data = df[f'peakparam_{channel}/peakparam_{channel}.charge']
        filters[channel].update({'charge_cut': (charge_data > min_charge_tot) & (charge_data < max_charge_tot)})

        ampl_data = df[f'peakparam_{channel}/peakparam_{channel}.ampl']
        filters[channel].update({'ampl_cut': (ampl_data > min_ampl) & (ampl_data < max_ampl)})

        dampl_data = df[f'peakparam_{channel}/peakparam_{channel}.dampl']
        filters[channel].update({'dampl_cut': (dampl_data > min_ampl) & (dampl_data < max_ampl)})
        filters[channel].update({'saturation_cut': dampl_data < mm_saturation_voltage})

        x_data = df[f'hitX_{channel}']
        y_data = df[f'hitY_{channel}']

        xy_filter_cut = (x_data > xy_range_min) & (x_data < xy_range_max) & (y_data > xy_range_min) & (y_data < xy_range_max)
        filters[channel].update({'xy_cut': xy_filter_cut})

    mm_charge_filtered_df = df[filters[mm_channel]['totcharge_cut']]

    df_filter = df[filters[mm_channel]['totcharge_cut'] & filters[mm_channel]['xy_cut'] & filters[mm_channel]['saturation_cut']]
    xs_mm_filtered = df_filter[f'hitX_{mm_channel}']
    ys_mm_filtered = df_filter[f'hitY_{mm_channel}']
    charges_mm_filtered = df_filter[f'peakparam_{mm_channel}/peakparam_{mm_channel}.totcharge']
    mm_pad_center_measures = get_pad_center(charges_mm_filtered, xs_mm_filtered, ys_mm_filtered, min_avg_charge_per_2d_bin=10, plot=True)
    print(f'{mm_channel} Pad Center: ({", ".join([str(xi) for xi in mm_pad_center_measures])})')

    df[f'hitR_{mm_channel}'] = np.sqrt((df[f'hitX_{mm_channel}'] - mm_pad_center_measures[0].val) ** 2 + (df[f'hitY_{mm_channel}'] - mm_pad_center_measures[1].val) ** 2)
    r_inner = 4.3
    r_outer = hex_radius_inner_to_outer(r_inner)
    filters[mm_channel].update({'r_cut': df[f'hitR_{mm_channel}'] < r_outer})

    time_walk_func = time_walk_double_exponential
    p0 = [1.0, -0.1, 0.5, -0.1, 4] #ampl1, decay1, amp2, decay2, bsl

    df_filter = df[filters[mm_channel]['totcharge_cut'] & filters[mm_channel]['xy_cut'] & filters[mm_channel]['r_cut'] & filters[mm_channel]['saturation_cut']]
    charges = df_filter[f'peakparam_{mm_channel}/peakparam_{mm_channel}.totcharge']
    time_diff = df_filter[f'{mm_channel}_time_diff']
    xs = df_filter[f'hitX_{mm_channel}']
    ys = df_filter[f'hitY_{mm_channel}']
    rs = df_filter[f'hitR_{mm_channel}']

    indiv_params, med_bins_params, gaus_bins_params = get_time_walk_parameterization(time_diff, charges, time_walk_func, p0, percentile_cut=(2, 98), plot=True, plot_indiv_fits=False)


